In [1]:
from subscription.enum import SubscriptionAccountType

In [2]:
instance = SubscriptionMember.objects.first()

In [4]:
accountType = SubscriptionAccountType.label(instance.subscription_account.type)

In [5]:
isCreateAccount = SubscriptionAccountType.label(accountType) == 'CREATE'

In [6]:
isCreateAccount

True

In [21]:
em = EmailManager(User.objects.get(email='test5@test5.com'))

In [22]:
em.invoice_to_dict()

0.0


{'items': [{'item_id': 'ii_1EHkUWGjkoMLHlzNQV9Z2yvc',
   'plan_description': 'Netflix - Individual Plan - Month of April',
   'prorated_description': 'Remaining time on Netflix - Individual Plan after 25 Mar 2019',
   'plan_amount': 6.99,
   'prorated_amount': 1.54,
   'was_refunded': False},
  {'item_id': 'ii_1EHkUOGjkoMLHlzNiz5TGcZT',
   'plan_description': 'Netflix - Individual Plan - Month of April',
   'prorated_description': 'Unused time on Netflix - Individual Plan after 25 Mar 2019',
   'plan_amount': -6.99,
   'prorated_amount': -1.54,
   'was_refunded': True},
  {'item_id': 'ii_1EHkSwGjkoMLHlzN2DrGJ1kf',
   'plan_description': 'Netflix - Individual Plan - Month of April',
   'prorated_description': 'Remaining time on Netflix - Individual Plan after 25 Mar 2019',
   'plan_amount': 6.99,
   'prorated_amount': 1.54,
   'was_refunded': False},
  {'item_id': 'ii_1EHkSrGjkoMLHlzNHD5138q3',
   'plan_description': 'Netflix - Individual Plan - Month of April',
   'prorated_description